<a href="https://colab.research.google.com/github/mkjubran/Fundamentals-of-AI-and-Machine-Learning/blob/main/EXPLORATORY_DATA_ANALYSIS_%E2%80%93_DATA_CLEANING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EXPLORATORY DATA ANALYSIS – DATA CLEANING

In this notebook, we will demonstrate Data Cleaning as part of Exploratory Data Analysis (EDA). We will work on a modified version of the cardiovascular dataset from Kaggle (https://www.kaggle.com/code/sulianova/eda-cardiovascular-data/data). The dataset consists of 70000 records of patient data in 12 features. The target class "cardio" equals 1, when a patient has cardiovascular disease, and it's 0 if a patient is healthy.

# Import Libraries

First, we need to import some libraries that will be used during data cleaning.

In [ ]:
import numpy as np
import pandas as pd

# Data Preparation

***Clone the dataset Repository***

The modified dataset can be cloned from the GitHub repository https://github.com/mkjubran/AIData.git as below

In [ ]:
!rm -rf ./AIData
!git clone https://github.com/mkjubran/AIData.git

***Read the dataset***

The data is stored in the cardio_train.csv file. Read the input data into a dataframe using the Pandas library (https://pandas.pydata.org/) to read the data.

In [ ]:
df = pd.read_csv("/content/AIData/cardio_train_modified.csv",sep=";")
df.head()

***Display Data Info and Check NAN***

To display the content of the data and type of features use the info() method

In [ ]:
df.info()

Here the dataframe consists of 70000 rows with 12 variables (features). Ten features are numerical and two features are objects (gender, smoke). We notice that for some of the features the number of non-null values does not equal 70000 which means that some feature values in the data are missing.

We can get the exact number of missing values for each feature using the isnull() method as below

In [ ]:
df.isnull().sum()

We can also get the number and percentage of patients' records that has one or more missing values

In [ ]:
print(df.isnull().any(axis=1).sum())
print(100*df.isnull().any(axis=1).sum()/df.shape[0],'%')

To display the records with NAN values

In [ ]:
df[df.isnull().any(axis=1)]

# Data Cleaning

**Data Cleaning: drop all empty records**

The first step is usually to drop all empty records. I.e. records with all features are NaN.

In [ ]:
df.dropna(how='all', inplace=True)
df.isnull().sum()

By comparing the number of NaN features before and after the last step, we notice that there were 3 empty records in the dataset. We notice also that the number of missing values for the features 'weight', 'ap_hi', ap_lo', and 'gluc' is very low. So the best choice is to delete these patients' records from the dataset.

**Data Cleaning: 'weight' feature**

List the patients' records with 'weight' feature is NaN

In [ ]:
df[df.weight.isnull()]

List the patients' records with 'weight' feature is not NaN

In [ ]:
df[df.weight.notna()]

Delete (drop) records with 'weight' feature is NaN be selecting only rows with weight does not equal to NaN.

In [ ]:
print(df.shape)
df.dropna(subset=['weight'], inplace=True)
print(df.shape)

In [ ]:
df.isnull().sum()

As can be observed the number of records in the data frame was reduced by 4 (69996) and there is no NAN value in the 'weight' feature

**Data Cleaning: 'ap_hi', ap_lo', and 'gluc' features**

We will do the same for the 'ap_hi', ap_lo', and 'gluc' features.

In [ ]:
print(df.shape)
df.dropna(subset=['ap_hi','ap_lo','gluc'], inplace=True)
print(df.shape)

In [ ]:
df.isnull().sum()

**Data Cleaning: 'gender' feature**

The gender feature is a string 'male, female' and we have many missing values. One option is to drop all records with 'gender' feature equals to 'NaN'. However this means dropping ~1.4% of the records and this is to be decided by the domain experts.

In [ ]:
dfgender = df.copy()
print(dfgender.isnull()['gender'].sum())
print(100*dfgender.isnull()['gender'].sum()/dfgender.shape[0],'%')
print(dfgender.shape)
dfgender.dropna(subset=['gender'], inplace=True)
print(dfgender.shape)

Another option is to replace all missing values in the 'gender' feature with the majority kind (male or female).

In [ ]:
df['gender'].value_counts()

In [ ]:
dfc = df.copy()
dfc['gender'].fillna(value='female', inplace=True)
dfc['gender'].value_counts()

As can be observed the number of female records increased.

A third option is to try to set the missing 'gender' feature values based on other values in the record. For example, we can check the correlation between 'gender' and 'height' features. 

In [ ]:
df[['gender','height']].apply(lambda x: x.factorize()[0]).corr()

It seems that there is not much correlation. Let us try to check with other features.

In [ ]:
df.apply(lambda x: x.factorize()[0]).corr()

It seems that the 'gender' feature has the highest correlation with the 'smoke' feature.

In [ ]:
df[['gender','smoke']].apply(lambda x: x.factorize()[0]).corr()

Let us explore the correlation using crosstab

In [ ]:
pd.crosstab(df['gender'],df['smoke'])

This implies that most non-smokers are females and most smokers are males in the dataset. So let us make all 'gender' feature with 'NaN values for smokers to be 'male', and all 'gender' feature with 'NaN values for non-smokers to be 'female'. 

In [ ]:
dfsmoke = df.copy()
dfsmoke.loc[(dfsmoke.gender.isnull()) & (dfsmoke['smoke'] == 'Yes'),'gender']='male'
dfsmoke.loc[(dfsmoke.gender.isnull()) & (dfsmoke['smoke'] == 'No'),'gender']='female'

Let us check the correlation using crosstab again.

In [ ]:
pd.crosstab(dfsmoke['gender'],dfsmoke['smoke'])

We observe that the number of female non-smokers increased and the male smokers increase also. We also need to check if there are still any 'NaN' values in the 'gender' feature. This could be because the 'smoke' feature has also NaN values.

In [ ]:
dfsmoke.isnull().sum()

There are 12 NaN values in the 'gender' feature. We will drop them because they make only very small percentage of the population (records in the dataset).

In [ ]:
print(dfsmoke.shape)
dfsmoke.dropna(subset=['gender'], inplace=True)
print(dfsmoke.shape)

In this notebook, we will consider the third option to deal with the 'NaN' values in the 'gender' feature.

In [ ]:
df = dfsmoke.copy()
df.isnull().sum()

**Data Cleaning: 'smoke' feature**

Now, for the 'smoke' feature, is there any correlation with the other features?

In [ ]:
df.apply(lambda x: x.factorize()[0]).corr()

Yes, there is a high correlation between the 'smoke' feature and both the 'gender' and 'alco' features. But since we already used the 'smoke' feature to deal with the NaN values in the 'gender' feature and thus the correlation between them might be affected, we will use the 'alco' feature to deal with the NaN values in the 'smoke' feature.

In [ ]:
pd.crosstab(df['smoke'],df['alco'])

We can observe from the crosstab results that most non-alcoholic persons in the dataset are non-smokers but alcoholic persons might or might not be smokers. So we will make all 'NaN' values in the 'smoke' feature for all records of non-alcoholic persons to be No. 

In [ ]:
df.loc[(df.smoke.isnull()) & (df['alco'] == 0.0),'smoke']='No'

Let us check the correlation using crosstab again.

In [ ]:
pd.crosstab(df['smoke'],df['alco'])

We observe that the number of non-alcoholic persons in the dataset is non-smokers increased. We will drop all other records with the 'smoke' feature equal to NaN.

In [ ]:
print(df.shape)
df.dropna(subset=['smoke'], inplace=True)
print(df.shape)
df.isnull().sum()

**Data Cleaning: 'height' feature**

Now, for the 'height' feature, is there any correlation with the other features?

In [ ]:
df.apply(lambda x: x.factorize()[0]).corr()

Yes, there is a high correlation between the 'height' feature and both the 'gender' and 'weight' features. However, the 'height' feature has a continuous value and we can not deal with it similar to the 'gender' feature'. Instead, we should create a model that predicts the 'height' feature based on the 'gender' and 'weight' features which we will study in the next modules. So, for now, we have two options, either to drop all records where the 'height' feature is NaN or replace all these NaN values with some statistical measure (mean, median) of the 'height' feature. In this notebook, we will replace the NaN values with the median of the values in the 'height' feature.

In [ ]:
print(df.height.median())
df['height'].fillna(df.height.median(), inplace=True)
print(df.height.median())
df.isnull().sum()


**Data Cleaning: 'cholesterol' feature**

To handle the NaN values in the 'cholesterol' feature, we will use the same method we used for the 'height' feature. However, because the values of the 'cholesterol' feature are between 0 and 3 (small range), we will use the mean instead of the median (the median will return an integer value).

In [ ]:
print(df.cholesterol.mean())
df['cholesterol'].fillna(df.cholesterol.mean(), inplace=True)
print(df.cholesterol.mean())
df.isnull().sum()


# Save Data

Now, we will save the clean dataset into a CSV file to be used in the next session.

In [ ]:
df.to_csv("/content/AIData/cardio_train_cleaned.csv",sep=";",index=False)

Check the '/content/AIData/' folder for the 'cardio_train_cleaned.csv' file and download it for future usage.